In [1]:
import yaml
import torch
import random

from torch.utils.data import Subset

from model.model import Model
from model.trainer import Trainer
from model.test import Test
from model.model_analyze import ModelAnalyze

from data_pre.dataset import TrainValidDataset, TestDataset
import pandas as pd


random_seed = 42
random.seed(random_seed)

pd.set_option('display.max_colwidth', None)

# Model & Tokenizer & Dataset 생성

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

with open('/root/upstage-nlp-summarization-nlp2/config.yaml', "r") as file:
    config = yaml.safe_load(file)

# Model & Tokenizer
model_instance = Model(config, device)
model = model_instance.getModel()
tokenizer = model_instance.getTokenizer()

# dataset
train_dataset = TrainValidDataset(config, tokenizer, is_train=True)
val_dataset = TrainValidDataset(config, tokenizer, is_train=False)
test_dataset = TestDataset(config, tokenizer)

/opt/conda/envs/myvenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# Trainer 학습

In [5]:
trainer = Trainer(config, model, train_dataset, val_dataset, tokenizer).get_trainer()
trainer.train()

/opt/conda/envs/myvenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/envs/myvenv/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,No log,1.614725,0.143275,0.027206,0.139412
2,4.245200,0.608342,0.361342,0.123474,0.346808
3,0.389600,0.580516,0.364083,0.128525,0.350191
4,0.325400,0.565599,0.370407,0.132611,0.354429
5,0.325400,0.560738,0.376468,0.139522,0.360432
6,0.295900,0.558430,0.379644,0.139568,0.363962
7,0.276600,0.560502,0.381270,0.144705,0.365656
8,0.259700,0.562498,0.374397,0.134737,0.359304
9,0.247200,0.565898,0.374683,0.138324,0.360761
10,0.247200,0.569009,0.379212,0.141993,0.363684


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

TrainOutput(global_step=4290, training_loss=0.746473709853379, metrics={'train_runtime': 1071.7407, 'train_samples_per_second': 232.463, 'train_steps_per_second': 7.278, 'total_flos': 4.177517883162624e+16, 'train_loss': 0.746473709853379, 'epoch': 11.0})

# Test 추론

In [3]:
model_path = '/root/upstage-nlp-summarization-nlp2/results/checkpoint-2340'
test_inst = Test(config, test_dataset, tokenizer, model_path, device)
result_df = test_inst()
result_df

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
100%|██████████| 16/16 [00:40<00:00,  2.53s/it]


,fname,summary
0,test_0,#Person1#은 더슨 씨에게 사무실 통신이 이메일 통신과 공식 메모로 제한된다고...
1,test_1,#Person2#는 교통 체증에 걸렸다. #Person1#는 #Person2#가 대...
2,test_2,케이트는 마샤와 히어로가 이혼하려고 한다고 #Person1#에게 말한다. #Pers...
3,test_3,#Person1#은 브라이언의 생일을 축하하기 위해 파티에 왔다. #Person2#...
4,test_4,#Person2#는 #Person1#에게 올림픽 스타디움에 5000개의 좌석이 있다...
...,...,...
494,test_495,찰리는 아빠를 데리러 가야하기 때문에 #Person1# 과 함께 비디오 게임을 하고...
495,test_496,#Person2#는 컨트리 음악에 관심을 가지게 된 계기와 라디오 방송국에서 일하기...
496,test_497,"앨리스는 #Person1#에게 기계를 어떻게 사용하는지 알려주고, 기계가 너무 많은..."
497,test_498,스티브가 매튜에게 전화를 걸어 계약이 다음 달에 끝나기 때문에 집을 찾고 있다고 말...


# Model 성능 분석

In [3]:
# train_dataset의 0.001
sample_size = int(len(train_dataset) * 0.001)
indices = random.sample(range(len(train_dataset)), sample_size)

valid_dataset = Subset(train_dataset, indices)
model_path = '/root/upstage-nlp-summarization-nlp2/results/checkpoint-2340'

In [7]:
model_analyze = ModelAnalyze(config, tokenizer, device)
result_df, result_mean = model_analyze.get_result(valid_dataset, model_path)
result_df = result_df.sort_values(by = 'rouge_mean')
result_df

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
100%|██████████| 1/1 [00:00<00:00,  1.12it/s]


,input_text,generated_text,label,rouge_1,rouge_2,rouge_l,rouge_mean
5,"#Person1# : 새로 생긴 헬스장에 가봤어? 정말 좋아.\n#Person2# : 나도 같은 말을 하려고 했어. 어제 오후에 거기에 갔었는데, 정말 인상적이었어.\n#Person1# : 그 웨이트 룸의 장비들, 상당한 비쌀 거라고 생각해.\n#Person2# : 내 말이 그 말이야. 내가 예전에 상업용 피트니스 시설에 가입했었는데, 거기에는 그런 가격의 장비들이 절반도 안 됐었어.\n#Person1# : 작년에 수수료가 올라갔을 땐 정말 화가 났었는데, 시설을 보니 돈이 잘 쓰였단 걸 인정해야겠어.\n#Person2# : 나도 동의해.\n#Person1# : 오, 완벽하게 갖춰진 아크로바틱 룸도 있더라.\n#Person2# : 오, 정말 재미있겠다. 나는 들어올 때 못 봤어.\n#Person1# : 나는 어떻게 사용해야 할지 모를 것들을 많이 봤어.\n#Person2# : 야! 원한다면 내가 기본 기술 몇 가지를 보여줄게.\n#Person1# : 와, 정말 좋을 것 같아. 먼저 수업에 등록해야 할지 궁금하네.\n#Person2# : 좋아! 얼른 다시 몸매 관리를 시작하고 싶어!","<usr>#Address# 과 #CarNumber# 는 새로 생긴 헬스장에 대해 이야기하고 있다. #Address# 은 웨이트 룸의 장비들이 비쌀 것이라고 생각하고, #Person5# 는 그것을 좋아한다. 그들은 기본 기술 몇 가지를 보여줄 것이다.</s>","#Person1# 와 #Person2# 모두 새 헬스장과 장비들에 감탄하며, 먼저 수업에 등록하기로 결정한다.</s>",0.0476,0.0000,0.0476,0.0317
4,"#Person1# : 샐리, 린다의 생일 선물로 뭘 사야 할지 모르겠어.\n#Person2# : 나도 그래. 오늘 오후에 나랑 같이 쇼핑 갈래, 마이크?\n#Person1# : 그래, 좋아. 그럼 나도 린다 선물을 고를 수 있겠네.\n#Person2# : 뭘 좋아할지 생각해봐야겠어. 린다는 취미가 뭐야?\n#Person1# : 음악 감상, 독서, 테니스를 좋아해.\n#Person2# : 우리 그럼 CD나 책을 선물할 수 있겠다. 테니스 라켓이나 테니스화도 괜찮을 것 같아.\n#Person1# : 좋아, 그럼 먼저 서점에 갔다가 백화점에 가자.\n#Person2# : 좋았어.",<usr> 마이크는 샐리에게 린다의 생일 선물로 CD나 책을 사달라고 제안한다. #Address# 은 먼저 서점에 가서 백화점에 가기로 결정한다.</s>,마이크와 샐리가 린다에게 줄 선물을 고르기 위해 함께 쇼핑 갈 예정입니다. 그들은 린다의 취향에 대해 이야기합니다.</s>,0.1111,0.0000,0.1111,0.0741
8,"#Person1# : 안녕하세요. 도와드릴 일이 있나요?\n#Person2# : 네, 다행히도. 영어를 하시나요?\n#Person1# : 스페인에 여행 오신 건가요?\n#Person2# : 네, 저는 뉴욕 출신인데 여기 사람들이 다 영어를 조금은 할 줄 알았어요.\n#Person1# : 그렇군요, 무슨 문제가 있나요?\n#Person2# : 심한 복통이 있어요. 밤새도록 잠을 못 잤어요. 그리고 지금은 머리도 너무 아파요.\n#Person1# : 알겠습니다. 먹은 것 때문에 그런 거 같나요?\n#Person2# : 아, 그런 것 같아요. 새우와 생선을 너무 많이 먹었어요. 맛있긴 했지만 제 몸에는 안 맞는 것 같아요.",<usr>#CarNumber# 는 #Address# 에게 스페인에서 심한 복통을 겪고 있다고 말합니다. #CarNumber# 가 먹은 것 때문에 그런 것 같다고 생각합니다.</s>,"#Person2# 는 스페인을 여행 중이고, #Person2# 는 드디어 영어를 할 줄 아는 사람을 만나 심한 배아픔을 치료받을 수 있었다.</s>",0.1622,0.0000,0.1622,0.1081
6,"#Person1# : 와주셔서 감사합니다.\n#Person2# : 천만에요. 무슨 문제인가요?\n#Person1# : 파이프가 새고 있어요.\n#Person2# : 파이프가 정확히 어디에 있나요?\n#Person1# : 화장실 벽 뒤에 있어요.\n#Person2# : 파이프가 샌 지는 얼마나 되었죠?\n#Person1# : 며칠 전부터 새기 시작했어요.\n#Person2# : 파이프에 접근하려면 저 벽을 뜯어내야 한다는 것을 알려드려야 해요.\n#Person1# : 알고 있어요.\n#Person2# : 문제가 되지 않을까요?\n#Person1# : 고쳐진다면 문제 없어요.\n#Person2# : 알겠습니다, 지금 작업을 시작하겠습니다.",<usr>#CarNumber# 는 #Address# 에게 화장실 벽 뒤에 있는 파이프가 며칠 전부터 새기 시작했다고 말합니다. #Person5# 는 그것이 문제가 되지 않을 것이라고 생각합니다.</s>,#Person2# 가 #Person1# 의 새는 파이프를 고치러 왔습니다. #Person1# 은 벽을 뜯어내는 것이 문제가 되지 않습니다.</s>,0.1579,0.0526,0.1579,0.1228
1,"#Person1# : 여기 F. O. B. 기준의 가격표가 있습니다. 가격은 계약금 없이 제공됩니다.\n#Person2# : 좋습니다, 실례할게요, 지금 바로 가격표를 살펴보겠습니다.\n#Person1# : 천천히 보세요.\n#Person2# : 한눈에 봐도 가격이 너무 높습니다.\n#Person1# : 그렇게 말씀하시다니 놀랍네요. 최근 몇 년 동안 생산 비용이 급등했다는 것을 알고 계시잖아요.\n#Person2# : 저희는 다른 업체와 비슷한 수준의 가격을 요구할 뿐입니다. 그게 합리적이지 않나요?\n#Person1# : 사업을 진행하기 위해, 가격을 어느 정도 양보하는 것도 고려할 수 있습니다. 하지만 먼저 주문하시고자 하는 수량에 대해 알려주셔야 그에 따라 저희가 가격을 조정할 수 있습니다.\n#Person2# : 주문 규모는 가격에 따라 달라집니다. 먼저 그 문제를 해결합시다.\n#Person1# : 말씀드렸듯이, 주문 규모가 충분히 크다면, 저희는 가격을 2퍼센트 낮출 준비가 되어 있습니다.\n#Person2# : 제가 가격이 너무 높다고 말할 때는, 그것은 단지 2 또는 3퍼센트 높다는 것을 의미하는 것이 아닙니다.\n#Person1# : 그럼 얼마나 높다는 건가요? 대략적으로 일려 주실 수 있나요?\n#Person2# : 이 사업을 마무리하기 위해서는, 적어도 10퍼센트의 인하가 도움이 될 것이라고 말할 수 있습니다.\n#Person1# : 불가능합니다. 어떻게 그 정도까지의 가격 인하를 기대하실 수 있나요?\n#Person2# : 화학 비료 시장에 대해 저만큼이나 잘 알고 계실 것입니다. 현재 공급이 수요를 초과하고 있으며, 이 상황이 앞으로도 오랫동안 지속될 가능성이 높다는 것을 지적할 필요는 없을 것 같습니다. 본사에 전화해서 그들의 의견을 들어보는 것이 어떨까요?\n#Person1# : 그럼, 그렇게 하겠습니다.",<usr>#CarNumber# 는 #Address# 에게 F. O. B. 기준의 가격표에 대해 설명합니다. #Person7# 는 가격을 2퍼센트 낮출 준비가 되어 있다고 말합니다.</s>,"#Person2# 는 #Person1# 이 제시한 가격이 너무 높다고 생각하고, #Person1# 은 가격에서 어느 정도 양보를 고려하고 있습니다. #Person1# 은 가격을 2퍼센트 낮추는 것을 제안하지만, #Person2# 는 적어도 10퍼센트 인하를 원합니다. #Person1# 은 타협할 의사가 없고, #Person2# 는 #Person1# 에게 본사에 전화홰 보는 것을 제안합니다. #Person1# 은 동의합니다.</s>",0.1818,0.0328,0.1818,0.1321
7,"#Person1# : 정말 대단한 영화였어요! 영화관에서 나오면서 이렇게 기분 좋은 적이 언제였는지 기억이 안 나네요.\n#Person2# : 제가 무슨 말인지 알아요. 로버트 레드포드는 제가 가장 좋아하는 배우니까, 제가 좋아할 거라는 건 알고 있었어요.\n#Pe